In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  1 23:16:54 2019

@author: RB
"""
"""
ARRAY will be used to hold the aggregated values for each hour
arrAggregateHourlyValues
12 rows (to be made 24 later) for each timeSlice of the overall period being analysed. Here 5 min slices temporarily (later 1 hour).
col 01: total trips - count
col 02: total passengers - summation
col 03: average passengers / trip - calculated value
col 04: store and forward flag=Y - count
col 05: store and forward flag=N - count
col 06: payment type - credit card (1) - count
col 07: payment type - cash        (2) - count
col 08: payment type - no charge   (3) - count
col 09: payment type - dispute     (4) - count
col 10: payment type - unknown     (5) - count
col 11: payment type - void        (6) - count
col 12: total fare amount - summation
col 13: average fare / trip - calculated value
col 14: total of trip distances - summations
col 15: average distance / trip - calculated value

LIST to hold aggregated hourly info: each of these elements repeats for each number of time slices being considered
        which is timeSliceNumber1 to timeSliceNumber24
each element [timeSliceNumber1,
              [
                [PULoc1,count], [PULoc2,count], [PULoc3, count] ,
                [DOLoc1,count], [DOLoc2,count], [DOLoc3, count] ,
                [PUDOLoc1,count], [PUDOLoc2,count], [PUDOLoc3, count], [PUDOLoc4, count], [PUDOLoc5, count]
              ] ,
              timeSliceNumber2,
              [
                [PULoc1,count], [PULoc2,count], [PULoc3, count] ,
                [DOLoc1,count], [DOLoc2,count], [DOLoc3, count] ,
                [PUDOLoc1,count], [PUDOLoc2,count], [PUDOLoc3, count], [PUDOLoc4, count], [PUDOLoc5, count]
              ] ,
              etc till timeSliceNumber24
             ]
"""
from pymongo import MongoClient
from datetime import datetime
import numpy as np

print('Started on test data with 10 docs and 12 timeslots spaced 5 mins apart covering one hour\nStartTime is:',datetime.now().strftime("%c"))

client = MongoClient('localhost:27017')
dbName = client.TestNYTFullJuneButOnlyFirst12Db1        # alternative way to code is:::  dbName = client['TwitterDb1']
collection = dbName.TestNYTFullJuneButOnlyFirst12Col1   # alternative way to code is:::  collection = dbName['TwitterCol1']

# verified that accessing any one element is a string e.g. timeSlotList[4] is a string
timeSlotList=["2018-06-01 00:00:00","2018-06-01 00:05:00","2018-06-01 00:10:00","2018-06-01 00:15:00",
              "2018-06-01 00:20:00","2018-06-01 00:25:00","2018-06-01 00:30:00","2018-06-01 00:35:00",
              "2018-06-01 00:40:00","2018-06-01 00:45:00","2018-06-01 00:50:00","2018-06-01 00:55:00",
              "2018-06-02 00:00:00"]


arrAggregateHourlyValues = np.zeros((12,15))
for idx1 in range(0,12):
    
    timeBoundLower = timeSlotList[idx1]
    timeBoundUpper = timeSlotList[idx1 + 1]
    
    #get total number of trips
    pipeline = [ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalTripsCount": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalTripsCount = collection.aggregate(pipeline)
    for cursor in cursorTotalTripsCount:
        arrAggregateHourlyValues[idx1,0] = cursor["TotalTripsCount"]
    cursorTotalTripsCount.close()
    
    #get total number of passengers
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalPassengers": {"$sum": "$passenger_count"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPassengers = collection.aggregate(pipeline)
    for cursor in cursorTotalPassengers:
        arrAggregateHourlyValues[idx1,1] = cursor["TotalPassengers"]
    cursorTotalPassengers.close()
    
    #get average number of passengers / trip
    arrAggregateHourlyValues[idx1,2] = arrAggregateHourlyValues[idx1,1] / arrAggregateHourlyValues[idx1,0]
    
    # total documents with Store and Forward Flag as YES
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"store_and_fwd_flag": "Y"}} , {"$group": {"_id": 1, "CountOfStoreFwdYES": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalStoreForwardYES = collection.aggregate(pipeline)
    for cursor in cursorTotalStoreForwardYES:
        arrAggregateHourlyValues[idx1,3] = cursor["CountOfStoreFwdYES"]
    cursorTotalStoreForwardYES.close()
    
    # total documents with Store and Forward Flag as NO
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"store_and_fwd_flag": "N"}} , {"$group": {"_id": 1, "CountOfStoreFwdNO": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalStoreForwardNO = collection.aggregate(pipeline)
    for cursor in cursorTotalStoreForwardNO:
        arrAggregateHourlyValues[idx1,4] = cursor["CountOfStoreFwdNO"]
    cursorTotalStoreForwardNO.close()
    
    # payment type - credit card (1)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 1}} , {"$group": {"_id": 1, "CountPaidByCreditCard": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentCreditCard = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentCreditCard:
        arrAggregateHourlyValues[idx1,5] = cursor["CountPaidByCreditCard"]
    cursorTotalPaymentCreditCard.close()
    
    # payment type - cash        (2)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 2}} , {"$group": {"_id": 1, "CountPaidByCash": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentCash = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentCash:
        arrAggregateHourlyValues[idx1,6] = cursor["CountPaidByCash"]
    cursorTotalPaymentCash.close()
    
    # payment type - no charge   (3)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 3}} , {"$group": {"_id": 1, "CountPaidByNoCharge": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentNoCharge = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentNoCharge:
        arrAggregateHourlyValues[idx1,7] = cursor["CountPaidByNoCharge"]
    cursorTotalPaymentNoCharge.close()
    
    # payment type - dispute     (4)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 4}} , {"$group": {"_id": 1, "CountPaidByDispute": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentDispute = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentDispute:
        arrAggregateHourlyValues[idx1,8] = cursor["CountPaidByDispute"]
    cursorTotalPaymentDispute.close()
    
    # payment type - unknown     (5)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 5}} , {"$group": {"_id": 1, "CountPaidByUnknown": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentUnknown = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentUnknown:
        arrAggregateHourlyValues[idx1,9] = cursor["CountPaidByUnknown"]
    cursorTotalPaymentUnknown.close()
    
    # payment type - void        (6)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 6}} , {"$group": {"_id": 1, "CountPaidByVoid": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentVoid = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentVoid:
        arrAggregateHourlyValues[idx1,10] = cursor["CountPaidByVoid"]
    cursorTotalPaymentVoid.close()
    
    #get total fare paid
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalOfFare": {"$sum": "$total_amount"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalFare = collection.aggregate(pipeline)
    for cursor in cursorTotalFare:
        arrAggregateHourlyValues[idx1,11] = cursor["TotalOfFare"]
    cursorTotalFare.close()
    
    #get average fare / trip
    arrAggregateHourlyValues[idx1,12] = arrAggregateHourlyValues[idx1,11] / arrAggregateHourlyValues[idx1,0]
    
    #get total trip distances
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalDistance": {"$sum": "$trip_distance"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalDistance = collection.aggregate(pipeline)
    for cursor in cursorTotalDistance:
        arrAggregateHourlyValues[idx1,13] = cursor["TotalDistance"]
    cursorTotalDistance.close()
    
    #get average distance / trip
    arrAggregateHourlyValues[idx1,14] = arrAggregateHourlyValues[idx1,13] / arrAggregateHourlyValues[idx1,0]
    

print(arrAggregateHourlyValues)
print('\nProgram Endtime is:',datetime.now().strftime("%c"))
#retDoc = colName.find_one()

#retDocUser = retDoc['user']

#print('\nDocument field created_at value is:', retDoc['created_at'])
#print('\nDocument field user.screen_name value is:', retDocUser['screen_name'])

#keys = retDoc.keys()

#print('\n\nFull document:',retDoc,'\n\n')

#keys_str = str(keys)

Started on test data with 10 docs and 12 timeslots spaced 5 mins apart covering one hour
StartTime is: Wed Jan  2 01:01:51 2019
[[ 2.     3.     1.5    0.     2.     2.     0.     0.     0.     0.
   0.    19.9    9.95   2.5    1.25 ]
 [ 1.     1.     1.     0.     1.     1.     0.     0.     0.     0.
   0.    15.35  15.35   2.     2.   ]
 [ 2.     2.     1.     0.     2.     1.     1.     0.     0.     0.
   0.    22.45  11.225  4.     2.   ]
 [ 2.     2.     1.     0.     2.     0.     2.     0.     0.     0.
   0.    11.6    5.8    0.85   0.425]
 [ 1.     1.     1.     0.     1.     0.     1.     0.     0.     0.
   0.    23.3   23.3    5.7    5.7  ]
 [ 0.     0.       nan  0.     0.     0.     0.     0.     0.     0.
   0.     0.       nan  0.       nan]
 [ 0.     0.       nan  0.     0.     0.     0.     0.     0.     0.
   0.     0.       nan  0.       nan]
 [ 0.     0.       nan  0.     0.     0.     0.     0.     0.     0.
   0.     0.       nan  0.       nan]
 [ 0.     0.    

C:\ProgramData\Anaconda3-5.3.1\lib\site-packages\ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3-5.3.1\lib\site-packages\ipykernel_launcher.py:170: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3-5.3.1\lib\site-packages\ipykernel_launcher.py:182: RuntimeWarning: invalid value encountered in double_scalars


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  1 23:16:54 2019

@author: RB
"""
"""
ARRAY will be used to hold the aggregated values for each hour
arrAggregateHourlyValues
12 rows (to be made 24 later) for each timeSlice of the overall period being analysed. Here 5 min slices temporarily (later 1 hour).
col 01: total trips - count
col 02: total passengers - summation
col 03: average passengers / trip - calculated value
col 04: store and forward flag=Y - count
col 05: store and forward flag=N - count
col 06: payment type - credit card (1) - count
col 07: payment type - cash        (2) - count
col 08: payment type - no charge   (3) - count
col 09: payment type - dispute     (4) - count
col 10: payment type - unknown     (5) - count
col 11: payment type - void        (6) - count
col 12: total fare amount - summation
col 13: average fare / trip - calculated value
col 14: total of trip distances - summations
col 15: average distance / trip - calculated value

LIST to hold aggregated hourly info: each of these elements repeats for each number of time slices being considered
        which is timeSliceNumber1 to timeSliceNumber24
each element [timeSliceNumber1,
              [
                [PULoc1,count], [PULoc2,count], [PULoc3, count] ,
                [DOLoc1,count], [DOLoc2,count], [DOLoc3, count] ,
                [PUDOLoc1,count], [PUDOLoc2,count], [PUDOLoc3, count], [PUDOLoc4, count], [PUDOLoc5, count]
              ] ,
              timeSliceNumber2,
              [
                [PULoc1,count], [PULoc2,count], [PULoc3, count] ,
                [DOLoc1,count], [DOLoc2,count], [DOLoc3, count] ,
                [PUDOLoc1,count], [PUDOLoc2,count], [PUDOLoc3, count], [PUDOLoc4, count], [PUDOLoc5, count]
              ] ,
              etc till timeSliceNumber24
             ]
"""
from pymongo import MongoClient
from datetime import datetime
import numpy as np

print('Started on slightly incorrect 24 hours of 01-June-2018 data with about 316k docs and 24 timeslots spaced 1 hour apart covering one day\nStartTime is:',datetime.now().strftime("%c"))

client = MongoClient('localhost:27017')
dbName = client.TestNYTFullJuneButOnly24hrsDb1        # alternative way to code is:::  dbName = client['TwitterDb1']
collection = dbName.TestNYTFullJuneButOnly24hrsCol1   # alternative way to code is:::  collection = dbName['TwitterCol1']

# verified that accessing any one element is a string e.g. timeSlotList[4] is a string
timeSlotList=["2018-06-01 00:00:00","2018-06-01 01:00:00","2018-06-01 02:00:00","2018-06-01 03:00:00",
              "2018-06-01 04:00:00","2018-06-01 05:00:00","2018-06-01 06:00:00","2018-06-01 07:00:00",
              "2018-06-01 08:00:00","2018-06-01 09:00:00","2018-06-01 10:00:00","2018-06-01 11:00:00",
              "2018-06-01 12:00:00","2018-06-01 13:00:00","2018-06-01 14:00:00","2018-06-01 15:00:00",
              "2018-06-01 16:00:00","2018-06-01 17:00:00","2018-06-01 18:00:00","2018-06-01 19:00:00",
              "2018-06-01 20:00:00","2018-06-01 21:00:00","2018-06-01 22:00:00","2018-06-01 23:00:00",
              "2018-06-02 00:00:00"]

arrAggregateHourlyValues = np.zeros((24,15))
for idx1 in range(0,24):
    
    timeBoundLower = timeSlotList[idx1]
    timeBoundUpper = timeSlotList[idx1 + 1]
    
    #get total number of trips
    pipeline = [ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalTripsCount": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalTripsCount = collection.aggregate(pipeline)
    for cursor in cursorTotalTripsCount:
        arrAggregateHourlyValues[idx1,0] = cursor["TotalTripsCount"]
    cursorTotalTripsCount.close()
    
    #get total number of passengers
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalPassengers": {"$sum": "$passenger_count"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPassengers = collection.aggregate(pipeline)
    for cursor in cursorTotalPassengers:
        arrAggregateHourlyValues[idx1,1] = cursor["TotalPassengers"]
    cursorTotalPassengers.close()
    
    #get average number of passengers / trip
    arrAggregateHourlyValues[idx1,2] = arrAggregateHourlyValues[idx1,1] / arrAggregateHourlyValues[idx1,0]
    
    # total documents with Store and Forward Flag as YES
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"store_and_fwd_flag": "Y"}} , {"$group": {"_id": 1, "CountOfStoreFwdYES": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalStoreForwardYES = collection.aggregate(pipeline)
    for cursor in cursorTotalStoreForwardYES:
        arrAggregateHourlyValues[idx1,3] = cursor["CountOfStoreFwdYES"]
    cursorTotalStoreForwardYES.close()
    
    # total documents with Store and Forward Flag as NO
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"store_and_fwd_flag": "N"}} , {"$group": {"_id": 1, "CountOfStoreFwdNO": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalStoreForwardNO = collection.aggregate(pipeline)
    for cursor in cursorTotalStoreForwardNO:
        arrAggregateHourlyValues[idx1,4] = cursor["CountOfStoreFwdNO"]
    cursorTotalStoreForwardNO.close()
    
    # payment type - credit card (1)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 1}} , {"$group": {"_id": 1, "CountPaidByCreditCard": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentCreditCard = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentCreditCard:
        arrAggregateHourlyValues[idx1,5] = cursor["CountPaidByCreditCard"]
    cursorTotalPaymentCreditCard.close()
    
    # payment type - cash        (2)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 2}} , {"$group": {"_id": 1, "CountPaidByCash": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentCash = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentCash:
        arrAggregateHourlyValues[idx1,6] = cursor["CountPaidByCash"]
    cursorTotalPaymentCash.close()
    
    # payment type - no charge   (3)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 3}} , {"$group": {"_id": 1, "CountPaidByNoCharge": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentNoCharge = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentNoCharge:
        arrAggregateHourlyValues[idx1,7] = cursor["CountPaidByNoCharge"]
    cursorTotalPaymentNoCharge.close()
    
    # payment type - dispute     (4)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 4}} , {"$group": {"_id": 1, "CountPaidByDispute": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentDispute = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentDispute:
        arrAggregateHourlyValues[idx1,8] = cursor["CountPaidByDispute"]
    cursorTotalPaymentDispute.close()
    
    # payment type - unknown     (5)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 5}} , {"$group": {"_id": 1, "CountPaidByUnknown": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentUnknown = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentUnknown:
        arrAggregateHourlyValues[idx1,9] = cursor["CountPaidByUnknown"]
    cursorTotalPaymentUnknown.close()
    
    # payment type - void        (6)
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$match": {"payment_type": 6}} , {"$group": {"_id": 1, "CountPaidByVoid": {"$sum": 1}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalPaymentVoid = collection.aggregate(pipeline)
    for cursor in cursorTotalPaymentVoid:
        arrAggregateHourlyValues[idx1,10] = cursor["CountPaidByVoid"]
    cursorTotalPaymentVoid.close()
    
    #get total fare paid
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalOfFare": {"$sum": "$total_amount"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalFare = collection.aggregate(pipeline)
    for cursor in cursorTotalFare:
        arrAggregateHourlyValues[idx1,11] = cursor["TotalOfFare"]
    cursorTotalFare.close()
    
    #get average fare / trip
    arrAggregateHourlyValues[idx1,12] = arrAggregateHourlyValues[idx1,11] / arrAggregateHourlyValues[idx1,0]
    
    #get total trip distances
    pipeline=[ {"$match": {"tpep_pickup_datetime": {"$gte": "2018-06-01 00:00:00", "$lt": "2018-06-01 01:00:00"}}} , {"$group": {"_id": 1, "TotalDistance": {"$sum": "$trip_distance"}}} ]
    pipeline[0]['$match']['tpep_pickup_datetime']['$gte'] = timeBoundLower
    pipeline[0]['$match']['tpep_pickup_datetime']['$lt'] = timeBoundUpper
    cursorTotalDistance = collection.aggregate(pipeline)
    for cursor in cursorTotalDistance:
        arrAggregateHourlyValues[idx1,13] = cursor["TotalDistance"]
    cursorTotalDistance.close()
    
    #get average distance / trip
    arrAggregateHourlyValues[idx1,14] = arrAggregateHourlyValues[idx1,13] / arrAggregateHourlyValues[idx1,0]
    

print(arrAggregateHourlyValues)
print('\nProgram Endtime is:',datetime.now().strftime("%c"))
#retDoc = colName.find_one()

#retDocUser = retDoc['user']

#print('\nDocument field created_at value is:', retDoc['created_at'])
#print('\nDocument field user.screen_name value is:', retDocUser['screen_name'])

#keys = retDoc.keys()

#print('\n\nFull document:',retDoc,'\n\n')

#keys_str = str(keys)



Started on slightly incorrect 24 hours of 01-June-2018 data with about 316k docs and 24 timeslots spaced 1 hour apart covering one day
StartTime is: Wed Jan  2 01:05:14 2019
[[1.00440000e+04 1.60890000e+04 1.60185185e+00 3.10000000e+01
  1.00130000e+04 7.25000000e+03 2.70500000e+03 6.90000000e+01
  2.00000000e+01 0.00000000e+00 0.00000000e+00 1.80292140e+05
  1.79502330e+01 3.70100300e+04 3.68478992e+00]
 [6.22400000e+03 9.97600000e+03 1.60282776e+00 3.10000000e+01
  6.19300000e+03 4.28900000e+03 1.86300000e+03 5.60000000e+01
  1.60000000e+01 0.00000000e+00 0.00000000e+00 1.11646780e+05
  1.79381073e+01 2.39609500e+04 3.84976703e+00]
 [3.85600000e+03 6.29300000e+03 1.63200207e+00 2.70000000e+01
  3.82900000e+03 2.43600000e+03 1.37500000e+03 3.50000000e+01
  1.00000000e+01 0.00000000e+00 0.00000000e+00 5.99943600e+04
  1.55587033e+01 1.25270600e+04 3.24871888e+00]
 [2.54200000e+03 3.97700000e+03 1.56451613e+00 1.50000000e+01
  2.52700000e+03 1.51800000e+03 9.84000000e+02 3.00000000e+01
